# House Prices - EDA and models comparison

*Autores: David Tejero Ruiz & Miguel Gil Castilla*


El [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) ha sido obtenido de la página web de Kaggle en el apartado de competiciones ...


In [ ]:
import pandas as pd
House_prices = pd.read_csv('data.csv')

In [ ]:
House_prices.head()


Decir que hay un montón y que hay que trabajar con los datos como se requiere en este tipo de trabajos

# Preprocesado de datos



In [ ]:
# Vamos a definir un problema de regresión sobre el precio de las casas, por lo que la variable 
# objetivo del dataset será 'SalePrice'
y_data = House_prices['SalePrice']

# Para tratar los diferentes atributos lo primero es distinguir entre las variables cuantitativas 
# (numéricas)y cualitativas (categóricas)

# Ya que el tipo de dato 'object' engloba a las variables categóricas y a las variables de tipo
# string, se puede usar dicho tipo de dato para distinguir las variables en numéricas y categóricas
numerical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] != 'object']
categorical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] == 'object']

# Eliminamos las variables 'SalePrice' (variable objetico) e 'Id' (no aporta información) de la
# lista de variables numéricas
numerical_atr.remove('SalePrice')
numerical_atr.remove('Id')
numerical_data = House_prices[numerical_atr]

# Para tratar las variables categóricas vamos a usar el método de 'one-hot encoding' que consiste
# en crear una nueva variable binaria por cada valor que pueda tomar la variable categórica.
# Para ello, usamos la función 'get_dummies' de pandas que efectúa dicho 'one-hot encoding' de
# forma automática
categorical_data = pd.get_dummies(House_prices[categorical_atr])

# Concatenamos las variables numéricas y las variables categóricas para obtener el dataset final
X_data = pd.concat([numerical_data, categorical_data], axis=1)

In [ ]:
X_data.head()

In [ ]:
print("Datos numéricos: ",numerical_atr)
print("Datos categóricos: ",categorical_atr)

## Vemos los valores perdidos de cada uno de los atributos

In [ ]:
perdidos = House_prices.isnull().sum()
perdidos = perdidos[perdidos > 0]

print("Hay ",len(perdidos),"atributos con valores perdidos")

# Mostramos a continuación una gráfica con los valores perdidos de cada atributo   
# (ordenados de mayor a menor)
import matplotlib.pyplot as plt

ordenados_perdidos = perdidos.sort_values(ascending=False)
plt.bar(ordenados_perdidos.index, ordenados_perdidos)
plt.xticks(rotation='vertical')
plt.show()

# Modelos

In [ ]:
# Dividimos los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=0)



COSAS QUE HAY QUE HACER:

**Preprocesado de datos**
- Eliminar columnas con muchos valores perdidos
- One-hot encoding / Label encoding (categorical variables)
- Normalizar datos: StandardScaler, MinMaxScaler, RobustScaler 
- Eliminar outliers: IQR, Z-score, etc.  (?)
- Discretizado de variables continuas (?): Binning, etc.


**Ingeneering features**
- Reducción de dimensionalidad: *PCA*, LDA, etc.
- Selección de variables - Filter Methods: Correlation, Chi2, ANOVA, etc.
- Multicolinealidad: VIF, etc. 
- Crear nuevas variables.

**Posibles Modelos**
- Regresión lineal
- Regresión polinómica
- Regresión Ridge
- Regresión Lasso
- Regresión ElasticNet
- SVR
- Random Forest
- **Gradient Boosting**
- XGBoost
- LightGBM
- CatBoost

**Evaluación de modelos**
- MSE
- k-fold cross validation
- Recall, Precision, F1-score, etc. (?)